In [16]:
# Import the required libraries
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


2023-03-02 20:27:57.623469: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 20:27:57.790237: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-02 20:27:57.790264: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-02 20:27:58.664724: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [ ]:
# Load the text data and preprocess it
filename = 'demo.txt'
text = open(filename).read().lower()
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
n_chars = len(text)
n_vocab = len(chars)
seq_length = 100

In [ ]:
# Prepare the input and output sequences for the LSTM model
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

X = np.reshape(dataX, (n_patterns, seq_length, 1))
X = X / float(n_vocab)
y = np_utils.to_categorical(dataY)

In [ ]:
# Define the LSTM model architecture
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))

# Compile the model and define the training parameters
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Define the checkpoint callback to save the model weights during training
filepath = 'weights-improvement-{epoch:02d}-{loss:.4f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# Train the LSTM model on the text data
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

In [ ]:
# Generate new text using the trained model
start_index = np.random.randint(0, len(dataX)-1)
pattern = dataX[start_index]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

for i in range(500):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x
